In [1]:
import os
import pandas as pd
import asyncio
from functools import partial
from twitterscraper import query_tweets
from IPython.core.debugger import set_trace

In [2]:
df = pd.read_excel('keywords.xlsx')[['item','keyword_eng','keyword_kr','intersection']].set_index('item')
_or = lambda kw: ' OR '.join(['#' + k.strip() for k in kw.split(',')])

df['keyword'] = df.keyword_eng + ', ' + df.keyword_kr
df['qry'] = df.keyword.apply(_or)

intersect_with = ['ootd', 'category', 'fashion']
screener = ' OR '.join(df.loc[intersect_with, 'qry'])
df.loc[df.intersection==True, 'qry'] = '(' + df.qry + ') AND (' + screener + ')'

qry = dict(df.qry)

In [3]:
def _period(start=None, end=None):
    if start is None: start = '2010-01-01'
    if end is None: end = pd.Timestamp.today()
    
    return pd.Timestamp(start).date(), pd.Timestamp(end).date()

In [26]:
def scrap(qry_dict, what=['id','text'], lang=None, start=None, end=None, download_to='db_tw'):
    start, end = _period(start=start, end=end)
    filename = download_to + '/{item} (' + str(start).replace('-','') + '-' + str(end).replace('-','') + ').pkl'
    
    if not os.path.exists(download_to):
        os.makedirs(download_to)    
    
    
    def _pretty(queried):
        return [{k:v for k,v in qrd.__dict__.items() if k in what} for qrd in queried]
    
    
    async def _scrap(item, q):
        q_tw_partial = partial(query_tweets, lang=lang, begindate=start, enddate=end)
        queried = await loop.run_in_executor(None, q_tw_partial, q)
        res = _pretty(queried)
        
        df = pd.DataFrame(res)
        df.to_pickle(filename.format(item=item))
    
    
    async def main():
        fts = [asyncio.ensure_future(_scrap(item, q)) for item, q in qry_dict.items()]
        await asyncio.gather(*fts)
    
    
    asyncio.set_event_loop(asyncio.new_event_loop())
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
    loop.close()

In [5]:
qry_test = {'aldo':qry['aldo'], 'allsaints':qry['allsaints']}

In [27]:
%%time
scrap(qry_test, lang='ko')

INFO: queries: ['#allsaints OR #올세인츠 since:2010-01-01 until:2010-06-15', '#allsaints OR #올세인츠 since:2010-06-15 until:2010-11-28', '#allsaints OR #올세인츠 since:2010-11-28 until:2011-05-13', '#allsaints OR #올세인츠 since:2011-05-13 until:2011-10-26', '#allsaints OR #올세인츠 since:2011-10-26 until:2012-04-09', '#allsaints OR #올세인츠 since:2012-04-09 until:2012-09-22', '#allsaints OR #올세인츠 since:2012-09-22 until:2013-03-06', '#allsaints OR #올세인츠 since:2013-03-06 until:2013-08-19', '#allsaints OR #올세인츠 since:2013-08-19 until:2014-02-01', '#allsaints OR #올세인츠 since:2014-02-01 until:2014-07-17', '#allsaints OR #올세인츠 since:2014-07-17 until:2014-12-30', '#allsaints OR #올세인츠 since:2014-12-30 until:2015-06-14', '#allsaints OR #올세인츠 since:2015-06-14 until:2015-11-27', '#allsaints OR #올세인츠 since:2015-11-27 until:2016-05-10', '#allsaints OR #올세인츠 since:2016-05-10 until:2016-10-23', '#allsaints OR #올세인츠 since:2016-10-23 until:2017-04-07', '#allsaints OR #올세인츠 since:2017-04-07 until:2017-09-20', '#allsaints OR 

ValueError: DataFrame constructor not properly called!

INFO: Got 341 tweets (124 new).


In [19]:
pd.read_pickle('db_tw/allsaints (20100101-20190131).pkl')

,id,text
0,208596651231817729,멋지다. 이 와중에 작동은 하나? 싶고.. RT @foreverkimchi: Sew...
1,394879338367418369,새롭게 론칭하는 슈즈 컬렉션 below the knee #올세인츠 pic.twitt...
2,385702682800824320,WORN IN LONDON. 최고급 캐시미어 니트와 레더 스커트를 통해 올세인츠가 ...
3,382463170624778242,고쳐야 저녁에 반짝반짝 불이 들어오지.. 아저씨들 고고 #올세인츠 pic.twitt...
4,377359278979293184,#올세인츠 #NYFW S14 프리젠테이션pic.twitter.com/ikJL5wDqq4
5,376804666035961856,발등에 피어싱이 장식된 시크한 로퍼! 올세인츠에서 가을부터 선보이는 슈즈 컬렉션 ...
6,376804230742691840,올세인츠의 프리젠테이션장에는 입고 싶은 룩 가득! 내년에 우리나라에서도 정식런칭된다...
7,375914648026873856,#올세인츠 가을 신상품 이 코트 정말 괜찮은듯 pic.twitter.com/RzLf...
8,750738188344303617,#프리드로우미리보기\nhttps://www.freestory01.com \n\n프...
9,744993307126730753,"#올세인츠 `KBS스페셜` 혁신 기업 넥스트 점프·올세인츠, 성공 요인은 바로 `..."
